In [1]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

# 코사인 유사도(Cosine Similarity)

$\text{Cosine Similarity}(A, B) = \frac{A \cdot B}{\|A\| \cdot \|B\|}$

$A \cdot B$ 는 벡터 A와 벡터 B의 내적을 나타난다.

$\|A\|$ 는 벡터 A의 크기를 나타낸다.
<br><br>


문서 단어 행렬에 대해서 코사인 유사도 예시
- 문서1 : 저는 사과 좋아요
- 문서2 : 저는 바나나 좋아요
- 문서3 : 저는 바나나 좋아요 저는 바나나 좋아요
<br><br>

띄어쓰기 기준 토큰화를 진행했다고 가정하고, 위의 세 문서에 대해서 문서 단어 행렬(DTM)을 만들면 이와 같습니다.

|       | 바나나 | 사과 | 저는 | 좋아요  |
|-------|-------|-----|-----|-------|
| 문서1  | 0     | 1   | 1   | 1     |
| 문서2  | 1     | 0   | 1   | 1     |
| 문서3  | 2     | 0   | 2   | 2     |

In [2]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :',cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


# 유사도를 이용한 영화 추천 시스템 구현하기

In [3]:
# kaggle 영화 데이터셋 다운로드 사이트
# https://www.kaggle.com/rounakbanik/the-movies-dataset

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
df = pd.read_csv('./data/cosine/movies_metadata.csv', low_memory=False)
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [6]:
# 상위 2만개의 샘플을 df에 저장
df = df.head(20000)

In [7]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 열의 결측값의 수:', df['overview'].isnull().sum())

overview 열의 결측값의 수: 135


In [8]:
# 결측값을 빈 값으로 대체
df['overview'] = df['overview'].fillna('')

In [9]:
# 20000개의 문서와 overview열에 대해 TF-IDF 행렬의 크기 : 20,000의 행을 가지고 47,847의 열을 가지는 행렬
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (20000, 47487)


In [10]:
# 20,000개의 각 문서 벡터(영화 줄거리 벡터)와 자기 자신을 포함한 20,000개의 문서 벡터 간의 유사도가 기록된 행렬
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [11]:
# 영화의 타이틀을 key, 영화의 인덱스를 value로 하는 딕셔너리 title_to_index를
title_to_index = dict(zip(df['title'], df.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [12]:
# 선택한 영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가 유사한 10개의 영화를 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = title_to_index[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [idx[0] for idx in sim_scores]

    return df['title'].iloc[movie_indices]

In [13]:
# 영화 다크 나이트 라이즈와 overview가 유사한 영화
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object